# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from new_api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
 # Load city weather data set into pandas
city_weather_data = pd.read_csv("../output_data/final_cities_weather.csv")
city_weather_data.head()


,name,lat,lon,temp,humidity,temp_max,clouds,wind,country,city_id,date
0,rikitea,-23.1203,-134.9692,77.83,68,77.83,87,7.58,PF,4030556,1643719764
1,arraial do cabo,-22.9661,-42.0278,82.26,78,82.36,75,3.44,BR,3471451,1643719765
2,busselton,-33.6500,115.3333,68.04,42,68.04,0,18.45,AU,2075265,1643719766
3,yellowknife,62.4560,-114.3525,-23.15,76,-22.45,20,9.22,CA,6185377,1643719727
4,hasaki,35.7333,140.8333,32.40,48,32.40,1,15.39,JP,2112802,1643719767


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key) 

locations = city_weather_data[["lat","lon"]].astype(float)

humidity_map = city_weather_data["humidity"].astype(float)


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
ideal_weather_df = city_weather_data.loc[(city_weather_data["temp_max"].between(75,90)) & (city_weather_data["wind"]<10) 
                                         &(city_weather_data["clouds"]==0)]
print(ideal_weather_df)

              name      lat       lon   temp  humidity  temp_max  clouds  \
12        santa fe -31.6333  -60.7000  76.37        54     78.01       0   
48        hermanus -34.4187   19.2345  85.80        75     85.80       0   
52         salalah  17.0151   54.0924  80.69        50     80.69       0   
53     tupancireta -29.0806  -53.8358  82.26        44     82.26       0   
100            sur  22.5667   59.5289  78.80        52     78.80       0   
156    east london -33.0153   27.9116  81.54        83     81.54       0   
307        guanica  17.9716  -66.9080  76.37        77     80.91       0   
375     vila velha -20.3297  -40.2925  82.94        80     82.94       0   
435  campo formoso -10.5075  -40.3214  81.57        53     81.57       0   
502     champerico  14.3000  -91.9167  73.90        78     75.04       0   
542         parana -31.7320  -60.5238  76.60        57     77.58       0   
552      nha trang  12.2500  109.1833  75.20        90     75.20       0   

     wind c

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:

# ideal_weather_df.head()
Hotel_df = ideal_weather_df.assign(Hotel = pd.NaT)

Hotel_df

,name,lat,lon,temp,humidity,temp_max,clouds,wind,country,city_id,date,Hotel
12,santa fe,-31.6333,-60.7000,76.37,54,78.01,0,5.01,AR,3836277,1643719637,NaT
48,hermanus,-34.4187,19.2345,85.80,75,85.80,0,9.86,ZA,3366880,1643719791,NaT
52,salalah,17.0151,54.0924,80.69,50,80.69,0,5.75,OM,286621,1643719760,NaT
53,tupancireta,-29.0806,-53.8358,82.26,44,82.26,0,6.33,BR,3445939,1643719717,NaT
100,sur,22.5667,59.5289,78.80,52,78.80,0,8.30,OM,286245,1643719823,NaT
156,east london,-33.0153,27.9116,81.54,83,81.54,0,8.05,ZA,1006984,1643719850,NaT
307,guanica,17.9716,-66.9080,76.37,77,80.91,0,1.99,PR,4565090,1643719933,NaT
375,vila velha,-20.3297,-40.2925,82.94,80,82.94,0,5.01,BR,3445026,1643719673,NaT
435,campo formoso,-10.5075,-40.3214,81.57,53,81.57,0,8.46,BR,3467760,1643719927,NaT
502,champerico,14.3000,-91.9167,73.90,78,75.04,0,3.22,GT,3598787,1643720041,NaT


In [10]:

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in Hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lon = row["lon"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    try:
        Hotel_df.loc[index, "Hotel"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
 
Hotel_df.head(10)  

,name,lat,lon,temp,humidity,temp_max,clouds,wind,country,city_id,date,Hotel
12,santa fe,-31.6333,-60.7000,76.37,54,78.01,0,5.01,AR,3836277,1643719637,InterTower Hotel
48,hermanus,-34.4187,19.2345,85.80,75,85.80,0,9.86,ZA,3366880,1643719791,Misty Waves Boutique Hotel
52,salalah,17.0151,54.0924,80.69,50,80.69,0,5.75,OM,286621,1643719760,Salalah Gardens Hotel
53,tupancireta,-29.0806,-53.8358,82.26,44,82.26,0,6.33,BR,3445939,1643719717,Hotel Diplomata
100,sur,22.5667,59.5289,78.80,52,78.80,0,8.30,OM,286245,1643719823,Sur Plaza Hotel
156,east london,-33.0153,27.9116,81.54,83,81.54,0,8.05,ZA,1006984,1643719850,Southern Sun Hemingways
307,guanica,17.9716,-66.9080,76.37,77,80.91,0,1.99,PR,4565090,1643719933,Copamarina Beach Resort & Spa
375,vila velha,-20.3297,-40.2925,82.94,80,82.94,0,5.01,BR,3445026,1643719673,Hotel Golden Tulip Porto Vitória
435,campo formoso,-10.5075,-40.3214,81.57,53,81.57,0,8.46,BR,3467760,1643719927,Hotel Rio das Pedras
502,champerico,14.3000,-91.9167,73.90,78,75.04,0,3.22,GT,3598787,1643720041,Hotel Miramar


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{name}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Hotel_df.iterrows()]
locations = Hotel_df[["lat", "lon"]]

In [12]:
hotel_layer=gmaps.symbol_layer(locations,fill_color='rgba(0,150,0,0.4)',scale=2, info_box_content=hotel_info)
fig=gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))